In [1]:
import json
import re
import random
import soundfile as sf
import numpy as np
import os
import mp
import polars as pl

In [2]:
sr = 16000
pattern = r"<\|([\d.]+)\|>"

In [3]:
!ls malaysian-stt/data/*.parquet

malaysian-stt/data/extra-00000-of-00001.parquet
malaysian-stt/data/imda-00000-of-00001.parquet
malaysian-stt/data/indonesian-00000-of-00001.parquet
malaysian-stt/data/malaysian_context-00000-of-00001.parquet
malaysian-stt/data/malaysian_context_v2-00000-of-00001.parquet
malaysian-stt/data/mandarin-00000-of-00001.parquet
malaysian-stt/data/science_context-00000-of-00001.parquet
malaysian-stt/data/science_en-00000-of-00001.parquet
malaysian-stt/data/science_ms-00000-of-00001.parquet
malaysian-stt/data/tamil-00000-of-00001.parquet


In [4]:
imda = pl.read_parquet('malaysian-stt/data/imda-00000-of-00001.parquet')
malay = pl.read_parquet('malaysian-stt/data/malaysian_context_v2-00000-of-00001.parquet')

In [5]:
imda[0]

audio_filename,word_timestamp,segment_timestamp
str,str,str
"""prepared-imda-chunks/0-0.mp3""","""<|startoftranscript|><|en|><|t…","""<|startoftranscript|><|en|><|t…"


In [6]:
!rm -rf /home/husein/ssd3/synthetic-context-switching-word-timestamp
!mkdir /home/husein/ssd3/synthetic-context-switching-word-timestamp

In [8]:
from tqdm import tqdm

selection = [malay, imda]
def loop(index):
    index, _ = index
    data = []
    for index_ in tqdm(index):
        for _ in range(5):
            chunk = []
            length = 0
            random_length = random.randint(10, 20)
            while length < random_length:
                x = random.choice(selection)
                s = random.choice(x)
                l = s['word_timestamp'][0].split('<|')[-2].split('|>')[0]
                length += float(l)
                chunk.append(s)

            audio = [chunk[0]['audio_filename'][0]]
            text = [chunk[0]['word_timestamp'][0].replace('<|endoftext|>', '')]
            silents = []
            for i in range(1, len(chunk), 1):
                matches = re.findall(pattern, text[-1])
                last_t = float(matches[-1])
                silent_l = random.randint(int(sr * (200 / 1000)), sr)
                t = chunk[i]['word_timestamp'][0].replace('<|endoftext|>', '').split('<|transcribeprecise|>')[1]
                matches = re.findall(pattern, t)
                for m in matches:
                    m_ = float(m)
                    m_ += (silent_l / sr) + last_t
                    t = t.replace(f'<|{m}|>', f'<|{m_:.2f}|>')
                silents.append(silent_l)
                text.append(t)
                audio.append(chunk[i]['audio_filename'][0])
                
            text = ''.join(text) + '<|endoftext|>'
            if len(text) > 4000:
                continue
            
            if float(text.split('<|')[-2].split('|>')[0]) > 30:
                continue
                
            audios = []
            for no, a in enumerate(audio):
                if 'prepared-pseudolabel-chunks' in a:
                    a = os.path.join('/home/husein/ssd3/', a)
                if 'prepared-imda-chunks'in a:
                    a = os.path.join('/home/husein/ssd3/', a)
                y, _ = sf.read(a)
                audios.append(y)

                if no < len(silents):
                    audios.append(np.zeros((silents[0],)))

            try:
                audios = np.concatenate(audios)
                f = os.path.join('/home/husein/ssd3/synthetic-context-switching-word-timestamp', f'{index_}.mp3')
                sf.write(f, audios, sr)
                data.append({
                    'word_timestamp': text,
                    'segment_timestamp': '',
                    'audio_filename': f,
                })
                break
            except:
                pass
        
        
    return data

In [9]:
r = loop((range(10), 0))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.10it/s]


In [10]:
r[-2]

{'word_timestamp': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> minta<|0.16|><|0.22|> maaf.<|0.70|><|1.08|> Saya<|1.18|><|1.22|> nak<|1.30|><|1.36|> tanya<|1.54|><|1.68|> ada<|1.80|><|1.88|> cadangan<|2.28|><|3.08|> untuk<|3.32|><|3.62|> mengadakan<|4.08|><|4.26|> Race<|4.46|><|4.50|> Relations<|4.86|><|4.94|> Act.<|5.04|><|5.50|> Adakah<|5.78|><|5.84|> Yang<|6.02|><|6.70|> know,<|6.80|><|6.86|> kids,<|7.14|><|7.90|> popo<|8.10|><|8.22|> come<|8.36|><|8.42|> to<|8.46|><|8.50|> your<|8.58|><|8.68|> door.<|8.86|><|9.28|> So<|9.44|><|10.00|> penjawat<|10.22|><|10.28|> awam<|10.50|><|10.58|> dijangka<|10.94|><|11.24|> akan<|11.46|><|11.52|> diambil<|11.86|><|11.88|> 2021<|12.50|><|12.68|> berapa<|12.92|><|13.74|> termasuklah<|14.20|><|14.26|> belangan<|14.54|><|14.58|> akan<|14.68|><|14.74|> pencin<|15.06|><|15.44|> ini<|15.56|><|15.74|> adalah<|15.92|><|16.30|> penting<|16.64|><|16.72|> untuk<|16.86|><|16.90|> anak<|17.02|><|17.08|> muda<|17.30|><|17.46|> yang<|17.62|><|17.78

In [11]:
import IPython.display as ipd
ipd.Audio(r[-2]['audio_filename'])

In [12]:
r = mp.multiprocessing(range(100000), loop, cores = 20)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [11:33<00:00,  7.21it/s]


In [17]:
r[-2]

{'word_timestamp': "<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> Kementerian<|0.52|><|0.62|> Pembangunan<|1.16|><|1.24|> Wanita,<|1.58|><|1.70|> Keluarga<|2.04|><|2.12|> dan<|2.22|><|2.28|> Masyarakat<|2.80|><|3.22|> serta<|3.52|><|3.88|> Institut<|4.36|><|4.58|> Pendidikan<|5.08|><|5.26|> Guru<|5.54|><|6.21|> I<|6.21|><|6.29|> decided<|6.81|><|6.89|> to<|6.97|><|7.17|> recreate<|7.91|><|8.09|> the<|8.15|><|8.21|> Victoria<|8.67|><|8.79|> sandwich<|9.37|><|9.64|> so<|9.72|><|10.02|> you<|10.08|><|10.16|> want<|10.30|><|10.34|> to<|10.36|><|10.50|> ask<|10.66|><|10.72|> first<|10.92|><|11.02|> or<|11.08|><|11.24|> I<|11.24|><|11.38|> ask<|11.64|><|11.87|> Okay,<|12.07|><|12.47|> so<|12.51|><|12.61|> that's<|12.77|><|12.79|> 10<|12.89|><|13.09|> portions<|13.53|><|13.71|> of<|13.87|><|14.13|> mango<|14.45|><|14.97|> Festival.<|15.47|><|15.61|> It's<|15.75|><|15.85|> literally<|16.27|><|16.45|> like<|16.75|><|16.87|> a<|16.87|><|16.97|> lechak<|17.29|><|17.35|> fest.<|17.55|><

In [16]:
ipd.Audio(r[-2]['audio_filename'])

In [15]:
import pandas as pd

pd.DataFrame(r).to_parquet('synthetic-context-switching-word-timestamp.parquet')